<a href="https://colab.research.google.com/github/zunaed-farabe/MH-project/blob/main/Meta_Heuristic_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.5 MB/s eta 0:00:00


# Loading data

In [2]:
def read_vrp_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    data = {}
    section = None
    for line in lines:
        line = line.strip()
        if not line:
            continue
        elif line.startswith('COMMENT'):
            pass
        elif line.startswith('NAME'):
            data['name'] = line.split(':')[1].strip()
        elif line.startswith('TYPE'):
            data['type'] = line.split(':')[1].strip()
        elif line.startswith('DIMENSION'):
            data['dimension'] = int(line.split(':')[1].strip())
        elif line.startswith('EDGE_WEIGHT_TYPE'):
            data['edge_weight_type'] = line.split(':')[1].strip()
        elif line.startswith('CAPACITY'):
            data['capacity'] = int(line.split(':')[1].strip())
        elif line.startswith('NODE_COORD_SECTION'):
            section = 'node_coords'
            data['node_coords'] = []
        elif line.startswith('DEMAND_SECTION'):
            section = 'demands'
            data['demands'] = []
        elif line.startswith('DEPOT_SECTION'):
            section = 'depots'
            data['depots'] = []
        elif section == 'node_coords':
            node_id, x, y = map(float, line.split())
            data['node_coords'].append((node_id, x, y))
        elif section == 'demands':
            node_id, demand = map(int, line.split())
            data['demands'].append((node_id, demand))
        elif section == 'depots':
            if line.startswith('EOF'):
              break
            else:
              node_id = int(line.split()[0])
              data['depots'].append(node_id)

    return data

file_path = '/content/gil262.vrp'
vrp_data = read_vrp_file(file_path)

# print(vrp_data)
# display(vrp_data["node_coords"][10:30])

In [19]:
type(vrp_data["node_coords"])

list

In [3]:
coordinates = []
for cord in vrp_data["node_coords"]:
  coordinates.append([cord[1], cord[1]])

array_coord = np.array(coordinates)
# Find the minimum value
min_val = np.min(array_coord)
print("Minimum value:", min_val)

# Find the maximum value
max_val = np.max(array_coord)
print("Maximum value:", max_val)

Minimum value: -99.0
Maximum value: 99.0


In [4]:
# Get the size of the original array
size = array_coord.shape[0]

# Create a square matrix of the same size filled with zeros
square_matrix = np.zeros((size, size))

# Assign the original values to their corresponding positions in the square matrix
for i, (x, y) in enumerate(array_coord):
    square_matrix[i, 0] = x
    square_matrix[i, 1] = y


print(square_matrix)

[[-33. -33.   0. ...   0.   0.   0.]
 [-99. -99.   0. ...   0.   0.   0.]
 [-59. -59.   0. ...   0.   0.   0.]
 ...
 [ 40.  40.   0. ...   0.   0.   0.]
 [-60. -60.   0. ...   0.   0.   0.]
 [-60. -60.   0. ...   0.   0.   0.]]


## Low Rank + (1+1) Evolutionary Strategy

In [12]:

from scipy.sparse.linalg import svds

def learn_low_rank_representation(X, s):
    """
    Learn a low-rank representation of the dataset X.

    Parameters:
        X (numpy.ndarray): The dataset.
        s (int): The number of rows/columns to sample.

    Returns:
        W (numpy.ndarray): The low-rank representation.
        residual_error (numpy.ndarray): The residual error.
    """
    # Random sampling of s rows/columns
    indices = np.random.choice(X.shape[0], s, replace=False)
    C = X[indices, :]
    R = X[:, indices]

    # Compute SVD of concatenated matrices
    U, sigma, Vt = svds(np.hstack((C.T, R)), k=s)

    # Extract the low-rank representation
    W = np.dot(U[:, :s], np.diag(sigma[:s]))

    # Calculate the residual error
    reconstructed_X = np.dot(W, Vt[:s, :])


    X = X.reshape(-1)  # [1, 2, 3, 4, 5, 6]
    reconstructed_X = reconstructed_X.reshape(-1)
    max_size = max(len(X), len(reconstructed_X))
    X_padded = np.pad(X, (0, max_size - len(X)), mode='constant')
    reconstructed_X_padded = np.pad(reconstructed_X, (0, max_size - len(reconstructed_X)), mode='constant')
    residual_error = X_padded - reconstructed_X_padded


    return W, residual_error

# Define the fitness function
def evaluate(individual,W,residual_error):


    individual = individual.reshape(-1)
    W = W.reshape(-1)
    max_size = max(len(W), len(individual))
    W_padded = np.pad(W, (0, max_size - len(W)), mode='constant')
    individual = np.pad(individual, (0, max_size - len(individual)), mode='constant')
    new_residual_error = W_padded.dot(individual)
    return np.sum(np.square(new_residual_error - residual_error))

def evolutionary_computation(W, residual_error, num_generations):
    """
    Perform evolutionary computation using the (1+1)-Evolution Strategy.

    Parameters:
        W (numpy.ndarray): The low-rank representation.
        residual_error (numpy.ndarray): The residual error.
        num_generations (int): The number of generations.

    Returns:
        final_solution (numpy.ndarray): The final solution found by the evolutionary strategy.
    """
    # Initialize the parent solution
    parent = np.random.uniform(0, 1, W.shape[1])

    # Perform (1+1)-ES
    for _ in range(num_generations):
        # Create an offspring by mutating the parent
        offspring = parent + np.random.normal(0, 0.1, W.shape[1])

        # Evaluate fitness of parent and offspring
        parent_fitness = evaluate(parent,W,residual_error)
        offspring_fitness = evaluate(offspring,W,residual_error)

        # Select the better solution for the next generation
        if offspring_fitness < parent_fitness:
            parent = offspring

    # The final solution is the parent after the last generation
    final_solution = parent

    return final_solution

def main(X, s, num_generations):
    """
    Main function to combine the steps of learning low-rank representation
    and performing evolutionary computation.

    Parameters:
        X (numpy.ndarray): The dataset.
        s (int): The number of rows/columns to sample.
        num_generations (int): The number of generations for evolutionary computation.

    Returns:
        final_solution (numpy.ndarray): The final solution found by the evolutionary computation.
    """
    # Step 1: Learning a low-rank representation
    W, residual_error = learn_low_rank_representation(X, s)

    # Step 2: Performing evolutionary computation
    final_solution = evolutionary_computation(W, residual_error, num_generations)

    return final_solution

# Example usage:
# X = np.random.rand(100, 100)
X = square_matrix
s = 2
num_generations = 20
result = main(X, s, num_generations)
print(result)

[-0.09446247 -0.01190985]


## Low Rank + PSO


In [15]:
from scipy.sparse.linalg import svds
import numpy as np

def learn_low_rank_representation(X, s):
    """
    Learn a low-rank representation of the dataset X.
    Parameters:
    X (numpy.ndarray): The dataset.
    s (int): The number of rows/columns to sample.
    Returns:
    W (numpy.ndarray): The low-rank representation.
    residual_error (numpy.ndarray): The residual error.
    """
    # Random sampling of s rows/columns
    indices = np.random.choice(X.shape[0], s, replace=False)
    C = X[indices, :]
    R = X[:, indices]
    # Compute SVD of concatenated matrices
    U, sigma, Vt = svds(np.hstack((C.T, R)), k=s)
    # Extract the low-rank representation
    W = np.dot(U[:, :s], np.diag(sigma[:s]))
    # Calculate the residual error
    reconstructed_X = np.dot(W, Vt[:s, :])
    X = X.reshape(-1)  # [1, 2, 3, 4, 5, 6]
    reconstructed_X = reconstructed_X.reshape(-1)
    max_size = max(len(X), len(reconstructed_X))
    X_padded = np.pad(X, (0, max_size - len(X)), mode='constant')
    reconstructed_X_padded = np.pad(reconstructed_X, (0, max_size - len(reconstructed_X)), mode='constant')
    residual_error = X_padded - reconstructed_X_padded
    return W, residual_error

# Define the fitness function
def evaluate(individual, W, residual_error):
    individual = individual.reshape(-1)
    W = W.reshape(-1)
    max_size = max(len(W), len(individual))
    W_padded = np.pad(W, (0, max_size - len(W)), mode='constant')
    individual = np.pad(individual, (0, max_size - len(individual)), mode='constant')
    new_residual_error = W_padded.dot(individual)
    return np.sum(np.square(new_residual_error - residual_error))

def pso(W, residual_error, num_particles, num_generations, c1, c2, w):
    """
    Perform Particle Swarm Optimization (PSO).
    Parameters:
    W (numpy.ndarray): The low-rank representation.
    residual_error (numpy.ndarray): The residual error.
    num_particles (int): The number of particles in the swarm.
    num_generations (int): The number of generations.
    c1 (float): The cognitive component.
    c2 (float): The social component.
    w (float): The inertia weight.
    Returns:
    best_solution (numpy.ndarray): The best solution found by PSO.
    """
    # Initialize particle positions and velocities
    particle_positions = np.random.uniform(0, 1, (num_particles, W.shape[1]))
    particle_velocities = np.zeros_like(particle_positions)

    # Initialize personal and global bests
    personal_bests = particle_positions.copy()
    personal_best_fitness = np.array([evaluate(p, W, residual_error) for p in personal_bests])
    global_best_index = np.argmin(personal_best_fitness)
    global_best = personal_bests[global_best_index]
    global_best_fitness = personal_best_fitness[global_best_index]

    # Perform PSO iterations
    for _ in range(num_generations):
        for i in range(num_particles):
            # Update velocity
            r1, r2 = np.random.uniform(0, 1, 2)
            cognitive_component = c1 * r1 * (personal_bests[i] - particle_positions[i])
            social_component = c2 * r2 * (global_best - particle_positions[i])
            particle_velocities[i] = w * particle_velocities[i] + cognitive_component + social_component

            # Update position
            particle_positions[i] += particle_velocities[i]

            # Update personal and global bests
            current_fitness = evaluate(particle_positions[i], W, residual_error)
            if current_fitness < personal_best_fitness[i]:
                personal_bests[i] = particle_positions[i]
                personal_best_fitness[i] = current_fitness
                if personal_best_fitness[i] < global_best_fitness:
                    global_best = personal_bests[i]
                    global_best_fitness = personal_best_fitness[i]

    best_solution = global_best
    return best_solution

def main(X, s, num_generations, num_particles=20, c1=2.0, c2=2.0, w=0.7):
    """
    Main function to combine the steps of learning low-rank representation and performing PSO.
    Parameters:
    X (numpy.ndarray): The dataset.
    s (int): The number of rows/columns to sample.
    num_generations (int): The number of generations for PSO.
    num_particles (int): The number of particles in the swarm.
    c1 (float): The cognitive component.
    c2 (float): The social component.
    w (float): The inertia weight.
    Returns:
    final_solution (numpy.ndarray): The final solution found by PSO.
    """
    # Step 1: Learning a low-rank representation
    W, residual_error = learn_low_rank_representation(X, s)

    # Step 2: Performing Particle Swarm Optimization
    final_solution = pso(W, residual_error, num_particles, num_generations, c1, c2, w)

    return final_solution

# Example usage:

X = square_matrix
s = 2
num_generations = 100
result = main(X, s, num_generations)
print(result)

[ 8.39268432e-01 -5.88608354e-05]
